In [3]:
import gym
env = gym.make('CartPole-v0')
n_states = env.observation_space.shape[0]
n_actions = env.action_space


2


/Users/bryanzhang/Library/Python/3.9/lib/python/site-packages/gym/envs/registration.py:555: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(


In [ ]:
rewards = []
ep_steps = []
for i_episode in rnage(1, )